<a href="https://colab.research.google.com/github/GarlandZhang/gans_in_action_notes/blob/master/mnsit_digit_gan(GANS_in_action).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 100
original_dim = 28 * 28
latent_dim = 2
intermediate_dim = 256
nb_epoch = 5
epsilon_std = 1.0

In [6]:
def sampling(args):
  z_mean, z_log_var = args
  epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.)
  return z_mean + K.exp(z_log_var / 2) * epsilon

In [38]:
# encoder
x = Input(shape=(original_dim,), name='input', dtype='float32')
h = Dense(intermediate_dim, activation='relu', name='encoding', dtype='float32')(x)
z_mean = Dense(latent_dim, name='mean', dtype='float32')(h) # learn mean of distribution of h values
z_log_var = Dense(latent_dim, name='log-variance', dtype='float32')(h) # learn variance of distribution of h values
z = Lambda(sampling, output_shape=(latent_dim,), dtype='float32')([z_mean, z_log_var]) # get randomized output with values based on z_mean, z_log_var distribution. why tho?
encoder = Model(x, [z_mean, z_log_var, z], name='encoder') 

In [43]:
# decoder
input_decoder = Input(shape=(latent_dim,), name='decoder_input', dtype='float32')
decoder_h = Dense(intermediate_dim, activation='relu', name='decoder_h', dtype='float32')(input_decoder)
x_decoded = Dense(original_dim, activation='sigmoid', name='flat_decoded', dtype='float32')(decoder_h) # why sigmoid?
decoder = Model(input_decoder, x_decoded, name='decoder')

In [44]:
output_combined = decoder(encoder(x)[2])
vae = Model(x, output_combined)
vae.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 784)               0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Model)              (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [45]:
def vae_loss(x, x_decoded_mean, z_log_var, z_mean, original_dim=original_dim):
  xent_loss = original_dim * objectives.binary_crossentropy(x, x_decoded_mean)
  kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
  return xent_loss + kl_loss

In [47]:
vae.compile(optimizer='adam', loss=vae_loss) # doesn't work because missing required args for vae_loss (z_log_var, z_mean)

TypeError: ignored

In [49]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255. # / 255. is standard normalization
x_test = x_test.astype('float32') / 255.

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [50]:
vae.fit(
    x_train,
    x_train, # goals is to get back original input by encoding the input to get gibberish, then decode it to get a similar image to original
    shuffle=True,
    nb_epoch=nb_epoch,
    batch_size=batch_size,
    validation_data=(x_test, x_test),
    verbose=1
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


RuntimeError: ignored